This notebook takes a look at the results from a hypothetical slot machine through the lens of probability
* Tutors:
  * Anthropic's AI Claude
  * Google's Gemini

Given a slot machine and five observed payouts

Observed numbers = [0,0,1.5,0,1.5]
  * Sum = 3
  * Average = 3/5 = 0.6
  * Sample size = 5



Methods used for developing probability estimates
* Guess 1 = discrete probability distribution, binomal distribution
* Guess 2 = continous probability distribution, normal distribution
* Guess 3 = monte carlo approach
* Guess 4 = bayseian approach

In [24]:
#Given a slot machine and five observed payouts

#Observed numbers = [0,0,1.5,0,1.5]
    #Sum = 3
    #Average = 3/5 = 0.6
    #Sample size = 5

#Methods used for developing proabability estimates
    #Guess 1 = discrete probability distribution, binomal distribution
    #Guess 2 = continous probability distribution, normal distribution
    #Guess 3 = monte carlo approach
    #Guess 4 = bayseian approach

import numpy as np
from scipy.stats import binom
from scipy import stats

def calc_mean(numbers):
    '''
    Manual mean calculator
    '''

    total = sum(numbers)
    pop = len(numbers)

    mean = total/pop

    return mean

def calc_median(numbers):
    """
    Manual median calculator

    Tutor: Anthropic's AI Claude
    """

    sorted_numbers = sorted(numbers)
    n = len(sorted_numbers)

    if n % 2 == 0:
        # If even length, average the two middle numbers
        return (sorted_numbers[n//2 - 1] + sorted_numbers[n//2]) / 2
    else:
        # If odd length, return the middle number
        return sorted_numbers[n//2]

def calc_standard_deviation(numbers):
    """
    Manual Standard Deviation Calculator

    Calculates the spread of the slot machine payouts

    Tutor: Anthropic's AI Claude
    """
    n = len(numbers)

    # Calculate mean
    mean = sum(numbers) / n

    # Calculate squared differences from mean
    squared_diff_sum = sum((x - mean) ** 2 for x in numbers)

    # Calculate variance and standard deviation
    variance = squared_diff_sum / n
    std_dev = variance ** 0.5

    return std_dev


#import numpy as np
#from scipy.stats import binom

def binomial_distribution(k, n, p):
    """
    Calculates the binomial probability mass function at point k
    The equation gives the probability of exactly k successes in n trials,
    where each trial has probability p of success.

    For the slot machine example with [0, 0, 1.5, 0, 1.5]:
    - n = 5 (total spins)
    - k = 2 (number of 1.5 payouts)
    - p = 0.4 (probability of getting 1.5, which occurred 2/5 times)
    Using these values would give us the probability of getting exactly 2 payouts of 1.5 in 5 spins.
    Uses the scipy.stats library for accurate calculation

    Parameters:
    k (int): Number of successes to calculate probability for
    n (int): Total number of trials
    p (float): Probability of success on each trial (between 0 and 1)

    Returns:
    float: Probability mass for exactly k successes

    Uses the numpy and scipy.stats libraries for accurate calculation

    Tutor: Anthropic's AI Claude

    Examples:
    >>> binomial_distribution(k=2, n=5, p=0.4)  # Probability of 2 successes in 5 trials with 0.4 probability each
    0.2304
    """
    # Using scipy's binom.pmf for accurate calculation
    # Alternative manual calculation would be:
    # probability = (np.math.comb(n, k)) * (p**k) * ((1-p)**(n-k))
    probability = binom.pmf(k, n, p)
    return probability

#import numpy as np
def normal_distribution(x, mu, sigma):
    """
    Calculates the normal distribution probability density at point x

    The equation gives the probability density at any point x for a distribution with specified mean and standard deviation.

    When we calculate the normal distribution probability density at the mean (x = 0.6), we get approximately 0.4843. This value represents
    the relative likelihood of observing a payout around 0.6 units if the payouts truly follow a normal distribution.

    Uses the numpy library

    Parameters:
    x (float): Point at which to evaluate the distribution
    mu (float): Mean of the distribution
    sigma (float): Standard deviation of the distribution

    Returns:
    float: Probability density at point x

    Tutor: Anthropic's AI Claude

    """
    nd = (1 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

    return nd

def bayesian_slot_probability(observed_data, prior_prob=0.4):
    """
    Bayesian approach to calculating the probability of winning on a slot machine
    based on observed data.

    Parameters:
    observed_data (list): List of observed payouts (e.g., [0, 0, 1.5, 0, 1.5])
    prior_prob (float): Prior probability of winning, defaults to 0.4 based on observed frequency

    Returns:
    float: Posterior probability of winning on the next spin

    The function uses Bayes' theorem:
    P(Win|Data) = P(Data|Win) * P(Win) / P(Data)

    For slot machines:
    - P(Win|Data) is our updated probability of winning after seeing the data
    - P(Data|Win) is probability of seeing our specific sequence given win probability
    - P(Win) is our prior belief about win probability (0.4 default)
    - P(Data) is probability of observing our sequence under any probability



    # Count wins (payouts > 0) and total observations
    wins = sum(1 for payout in observed_data if payout > 0)
    total_spins = len(observed_data)

    # Calculate likelihood: P(Data|Win)
    # This is the probability of observing this many wins given our prior
    likelihood = (prior_prob ** wins) * ((1 - prior_prob) ** (total_spins - wins))

    # Calculate marginal probability: P(Data)
    # For simplification, we can use the observed frequency
    marginal = wins / total_spins

    # Calculate posterior probability using Bayes' theorem
    posterior_prob = (likelihood * prior_prob) / marginal if marginal > 0 else prior_prob

    # Ensure probability is between 0 and 1
    posterior_prob = min(1.0, max(0.0, posterior_prob))


    P(A|B) = (P(B|A) * P(A)) / P(B)

    P(A|B) = Posterior probability
    P(B|A) = Likelihood
    P(A) = Prior probability
    P(B) = Marginal probability

    Tutors:
      Google's AI Gemini
      Anthropic's AI Claude

    """
    # Count wins (payouts > 0) and total observations
    wins = sum(1 for payout in observed_data if payout > 0)
    total_spins = len(observed_data)

    # Use the Beta distribution conjugate prior
    # Beta(α + wins, β + losses) where α=β=1 for uniform prior
    alpha = 1 + wins
    beta = 1 + (total_spins - wins)

    # Posterior probability is the expected value of the Beta distribution
    posterior_prob = alpha / (alpha + beta)

    return posterior_prob


def monte_carlo_simulation(observed_data, n_simulations=10000, n_spins=5):
    """
    Monte Carlo simulation for slot machine outcomes based on observed data.

    Parameters:
    observed_data (list): List of observed payouts (e.g., [0, 0, 1.5, 0, 1.5])
    n_simulations (int): Number of simulations to run
    n_spins (int): Number of spins per simulation

    Returns:
    dict: Dictionary containing:
        - average_payout: Mean payout across all simulations
        - win_probability: Probability of winning (getting any payout > 0)
        - confidence_interval: 95% confidence interval for win probability
        - payout_distribution: Dictionary of payout frequencies


    E(x) = Σ(x_i * p_i) / Σ(p_i)

    E(x) = Expected value
    x_i = Individual values
    p_i = Probabilities for each value

    E = 0
    for i in range(len(numbers)):
        E += numbers[i] * p[i]

    return E

    Python libraries
      import numpy as np
      from scipy import stats

    Tutors:
      Google's AI Gemini
      Anthropic's AI Claude

    """

    # Get unique payouts and their frequencies from observed data
    unique_payouts = np.unique(observed_data)
    payout_probs = [np.mean(np.array(observed_data) == payout) for payout in unique_payouts]

    # Run simulations
    simulation_results = []
    win_counts = []

    for _ in range(n_simulations):
        # Generate sequence of spins
        spins = np.random.choice(unique_payouts, size=n_spins, p=payout_probs)

        # Record results
        simulation_results.append(np.mean(spins))
        win_counts.append(np.sum(spins > 0) / n_spins)

    # Calculate statistics
    average_payout = np.mean(simulation_results)
    win_probability = np.mean(win_counts)

    # Calculate 95% confidence interval
    confidence_interval = stats.norm.interval(
        0.95,
        loc=win_probability,
        scale=stats.sem(win_counts)
    )

    # Calculate payout distribution
    payout_distribution = {
        float(payout): len([x for x in simulation_results if x == payout]) / n_simulations
        for payout in unique_payouts
    }

    return {
        'average_payout': average_payout,
        'win_probability': win_probability,
        'confidence_interval': confidence_interval,
        'payout_distribution': payout_distribution
    }


def estimate_average_slot_payout(n_runs):
    """Run the slot machine n_runs times and return the average net profit per run.

    Example calls (note that return value is nondeterministic!):

    Tutors:
      Google's Gemini
      Anthropic's AI Claude

    """
    numbers = [0,0,1.5,0,1.5]

    sum = 0
    for i in range(n_runs):
        sum += sum(numbers)

    total_profit = 0
    for i in range(n_runs):
        total_profit += sum(numbers)

    pop = len(numbers)

    total_profit = sum(numbers)/pop
    average_profit = total_profit / n_runs

    return average_profit



# Observed results
numbers = [0,0,1.5,0,1.5]
total_payout = sum(numbers)
average_payout = total_payout / len(numbers)

print("\nInitial Data Analysis:")
print(f"Observed results: {numbers}")
print(f"Sample size: {len(numbers)}")
print(f"Observed wins: {sum(1 for x in numbers if x > 0)} out of {len(numbers)} spins")
print(f"Average Payout: ${average_payout:.3f}\n")

mean = calc_mean(numbers)
print(f"Mean: {mean}")
median = calc_median(numbers)
print(f"Median: {median}")
StanDev = calc_standard_deviation(numbers)
print(f"Standard deviation: {StanDev}\n")

# Enhanced Binomial Distribution Output
k = 2  # number of 1.5 payouts
n = 5  # total spins
p = 0.4  # probability of getting 1.5, which occurred 2/5 times
bin_prob = binomial_distribution(k, n, p)
expected_payout = p * 1.5  # Expected payout per spin

print("Binomial Distribution Results:")
print(f"Average Payout: ${expected_payout:.3f}")
print(f"Probability of exactly {k} wins in {n} spins: {bin_prob:.3f}")
print(f"Parameters: n={n} trials, p={p:.2f} success probability")
print(f"Expected number of wins: {n*p:.2f}")
print(f"Distribution type: Discrete probability (success/failure pattern)")
print("Note: Binomial distribution is appropriate for this slot machine scenario")
print("      as outcomes are binary (win/no win) with fixed probability.\n")

# Enhanced Normal Distribution Output
x = mean
mu = median
sigma = StanDev
norm_prob = normal_distribution(x, mu, sigma)
normal_expected_payout = mu  # Expected payout under normal distribution

print("Normal Distribution Results:")
print(f"Average Payout: ${normal_expected_payout:.3f}")
print(f"Probability density at mean (x={x:.2f}): {norm_prob:.3f}")
print(f"Distribution parameters: μ={mu:.2f}, σ={sigma:.2f}")
print(f"68% of values fall between: ({mu-sigma:.2f}, {mu+sigma:.2f})")
print(f"95% of values fall between: ({mu-2*sigma:.2f}, {mu+2*sigma:.2f})")
print("Note: Normal distribution is less appropriate here due to")
print("      the discrete nature of slot machine outcomes.\n")

# Monte Carlo Output
results = monte_carlo_simulation(numbers)
print("Monte Carlo Simulation Results:")
print(f"Average Payout: ${results['average_payout']:.3f}")
print(f"Win Probability: {results['win_probability']:.3f}")
print(f"95% Confidence Interval: ({results['confidence_interval'][0]:.3f}, {results['confidence_interval'][1]:.3f})")
print("\nPayout Distribution:")
for payout, prob in results['payout_distribution'].items():
    print(f"${payout}: {prob:.3f}")
print()

# Enhanced Bayesian Output
posterior_probability = bayesian_slot_probability(numbers)
bayesian_expected_payout = posterior_probability * 1.5  # Expected payout using posterior probability

print("Bayesian Analysis Results:")
print(f"Average Payout: ${bayesian_expected_payout:.3f}")
print(f"Posterior probability of winning: {posterior_probability:.3f}")
print(f"Prior probability used: 0.400")
print(f"Observed win rate: {sum(1 for x in numbers if x > 0)/len(numbers):.3f}")
print(f"Updated belief after {len(numbers)} observations")
print("Note: Using Beta distribution as conjugate prior for")
print("      binary outcome probability estimation.\n")

# Summary of all approaches
print("Summary of Average Payouts Across Methods:")
print(f"Observed Data Average:     ${average_payout:.3f}")
print(f"Binomial Expected:         ${expected_payout:.3f}")
print(f"Normal Distribution Mean:  ${normal_expected_payout:.3f}")
print(f"Monte Carlo Simulated:     ${results['average_payout']:.3f}")
print(f"Bayesian Expected:         ${bayesian_expected_payout:.3f}")


Initial Data Analysis:
Observed results: [0, 0, 1.5, 0, 1.5]
Sample size: 5
Observed wins: 2 out of 5 spins
Average Payout: $0.600

Mean: 0.6
Median: 0
Standard deviation: 0.7348469228349535

Binomial Distribution Results:
Average Payout: $0.600
Probability of exactly 2 wins in 5 spins: 0.346
Parameters: n=5 trials, p=0.40 success probability
Expected number of wins: 2.00
Distribution type: Discrete probability (success/failure pattern)
Note: Binomial distribution is appropriate for this slot machine scenario
      as outcomes are binary (win/no win) with fixed probability.

Normal Distribution Results:
Average Payout: $0.000
Probability density at mean (x=0.60): 0.389
Distribution parameters: μ=0.00, σ=0.73
68% of values fall between: (-0.73, 0.73)
95% of values fall between: (-1.47, 1.47)
Note: Normal distribution is less appropriate here due to
      the discrete nature of slot machine outcomes.

Monte Carlo Simulation Results:
Average Payout: $0.600
Win Probability: 0.400
95% Conf